In [1]:
mutable struct StackmanModel
    
    # utility function 
    β::Float64
    γ::Float64

    # stochastic process
    μ::Float64
    σ::Float64
    
end

In [2]:
function StackmanModel(;μ = 0.2, γ = 2., σ = 0.36, β = 0.95)
    StackmanModel(β, γ, μ, σ)
end

StackmanModel

In [3]:
function initialize_state(m::StackmanModel; m_gird_length = 100, λ_gird_length = 100)
    OrderedDict(:m => linspace(-1., 2., m_gird_length), 
        :λ => linspace(0., 1., λ_gird_length))
end

function initialize_y(m::StackmanModel, state)
    OrderedDict(:v => fill(1.0, length(state[:m]), length(state[:λ])))
end

initialize_y (generic function with 1 method)

In [4]:
function (m::StackmanModel)(state, y)
    #unpack constants, state, current solution
    β = m.β; γ = m.γ; μ = m.μ; σ = m.σ
    m = state.m; λ = state.λ
    v = y.v; vm = y.vm; vmm = y.vmm; vλ = y.vλ; vλλ = y.vλλ; vmλ = y.vmλ
    
    # drift
    μm = λ * (μ - m)
    μλ = - λ^2
    
    # PDE
    vt = μ + σ^2 / 2 * (1 - γ) 
        - β * v - vλ * λ^2
        + vm * λ * (μ - m + (1 + γ) * σ^2)
        + vm^2 * λ^2 * σ^2 / 2 * (1 - γ)
        + vmm * λ^2 * σ^2 / 2
    
    return vt, (μm, μλ), tuple(:v => v)
end

In [5]:
using EconPDEs # install: Pkg.clone("https://github.com/matthieugomez/EconPDEs.jl")
m = StackmanModel()
state = initialize_state(m)
y0 = initialize_y(m, state);

In [8]:
y, result, distance = pdesolve(m, state, y0; iterations = 5)

(iter, Δ, distance) = (1, 1.0, 0.0013520000000000001)
(iter, Δ, distance) = (2, 2.0, 0.0013520000000000001)
(iter, Δ, distance) = (3, 8.0, 0.0013520000000000001)
(iter, Δ, distance) = (4, 64.0, 0.0013520000000000001)
(iter, Δ, distance) = (5, 1024.0, 0.0013520000000000001)
(iter, Δ, distance) = (6, 32768.0, 0.0013520000000000001)


(DataStructures.OrderedDict(:v=>[4579.82 4579.82 … 4579.82 4579.82; 4579.82 4579.82 … 4579.82 4579.82; … ; 4579.82 4579.82 … 4579.82 4579.82; 4579.82 4579.82 … 4579.82 4579.82]), DataStructures.OrderedDict(:v=>[4579.82 4579.82 … 4579.82 4579.82; 4579.82 4579.82 … 4579.82 4579.82; … ; 4579.82 4579.82 … 4579.82 4579.82; 4579.82 4579.82 … 4579.82 4579.82]), 0.0013520000000000001)

In [9]:
y

DataStructures.OrderedDict{Symbol,Array{Float64,2}} with 1 entry:
  :v => [4579.82 4579.82 … 4579.82 4579.82; 4579.82 4579.82 … 4579.82 4579.82; …